# What is a Database?

  > a structured set of data held in a computer, especially one that is accessible in various ways. a database covering nine million workers. [ as modifier ] : database systems.
  >
  > _Oxford Dictionary of English_

  > a usually large collection of data organized especially for rapid search and retrieval (as by a computer)
  >
  > https://www.merriam-webster.com/dictionary/database

# What is a Database Management System?

  > A database-management system (DBMS) is a computer-software application that interacts with end-users, other applications, and the database itself to capture and analyze data. A general-purpose DBMS allows the definition, creation, querying, update, and administration of databases.
  > 
  > https://en.wikipedia.org/wiki/Database

# Your turn!

![](http://www.twenty19.com/blog/wp-content/uploads/2017/07/typing2.gif)

Build the likely simplest DB in the world!

# What is this?

```bash
#!/usr/bin/env bash
db_set () {
    echo "$1,$2" >> database
}
db_get () {
    grep "^$1," database | sed -e "s/^$1,//" | tail -n 1
}
```

  * Type in the script above using the editor of your choice.
  * **OBS** keep the spaces as they are, some of them are important.
  * Save the script above in a file called `simple_db.sh`. 
  * Then source this file via `source simple_db.sh`.

## Using the `simple_db`


Run the following command subsequently and see what happens.

```bash
$ db_set 123456 '{"name":"London","attractions":["Big Ben","London Eye"]}' 
$ db_set 42 '{"name":"San Francisco","attractions":["Golden Gate Bridge"]}'
$ db_get 42
```

## Exercise 1

  * Talk to your neighbour and:
    - Explain each other what the script -the DBMS `simple_db.sh`- does when _writing_ a record.
    - Explain each other what the script -the DBMS `simple_db.sh`- does when _reading_ a record.

  * What are really good features of our `simple_db.sh` DB?
  * Additionally, create a list of three potential issues of our simple database.

### Performance Issue

  * Finding a datum for a key takes $O(n)$, i.e., the worst case run time is linear.
  * What does that mean?

In [18]:
def inpect_each_element_in_a_list(list_len):
    for el in range(list_len):
        if el == 'a':
            print(f'Found: {el}')


%timeit inpect_each_element_in_a_list(1000000)

46.6 ms ± 498 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
%timeit inpect_each_element_in_a_list(1000000 * 2)

97.3 ms ± 3.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [21]:
%timeit inpect_each_element_in_a_list(1000000 * 10)

472 ms ± 15.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### How can we speed-up the look up?

  * Can we speed up the look up operation to constant time, i.e., $O(1)$
  * Which data structure do you know in your favorite programming language that supports constant time look up?

#### Hashmaps/Dictionaries in ...

  * Go
    ```golang
    m := map[string]int{"ananas": 3, "banana": 8,    "cranberrry": 4909}
    delete(m, "banana")
    m["banana"] = 42
   ```

  * Java
    ```java
Map<Integer, Integer> hm = new HashMap<Integer, Integer>();
```
  * C`#`
    ```c#
Dictionary<int, int> hm = new Dictionary<int, int>();
```
  * Python
    ```
    m = {"Hej": "Ho", "Let's": "Go!"}
    ```

In [11]:
import random
import string


def generate_random_data():
    chars = string.ascii_letters + string.digits
    return ''.join(random.sample(chars, 5))


list_len = 10
simple_index = {el: generate_random_data() for el in range(list_len)}
simple_index

{0: 'sH0vr',
 1: 'woUYy',
 2: 'DlcQ1',
 3: 'z0RcZ',
 4: 't1fcs',
 5: 'HQSNl',
 6: 'o5E9g',
 7: '3Lda1',
 8: 'is9CS',
 9: 'M4P6U'}

In [25]:
list_len = 1000000 * 2
simple_index = {el: generate_random_data() for el in range(list_len)}

In [26]:
%timeit simple_index[10000]

51.9 ns ± 0.435 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [24]:
%timeit simple_index[10000 * 2]

52.6 ns ± 0.428 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [15]:
%timeit simple_index[10000 * 50]

51.9 ns ± 0.525 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


### We just built an _index_!

  * What is the drawback of using a Hashmap as a datastructure for keeping a database index?
  * Talk to your neighbour and name two potential drawbacks.

# Why does this matter?

  * One of the oldest DBMS -the Unix library `dbm`- works in that way, see in the following.
  * Modern **key-value stores** such as _Riak_ (http://basho.com/products/riak-kv/) with the storage engine _Bitcask_ (https://docs.basho.com/riak/kv/2.0.7/setup/planning/backend/bitcask/) work that way.


## A closer look at `gdbm`, the _GNU dbm_

Let's say you have a program in some language -here Ruby- and you want to persist some _key-value_ data.

Save it in `gdbm_write.rb`.

```ruby
require 'gdbm'

gdbm = GDBM.new("fruitstore.db")
gdbm["ananas"]    = "3"
gdbm["banana"]    = "8"
gdbm["cranberry"] = "4909"
gdbm["ananas"]    = "42"
gdbm.close
```

The example is adapted from:
https://ruby-doc.org/stdlib-2.5.0/libdoc/gdbm/rdoc/GDBM.html

```ruby
require 'gdbm'

gdbm = GDBM.new("fruitstore.db")
gdbm.each_pair do |key, value|
  print "#{key}: #{value}\n"
end
gdbm.close
```

In case you have no Ruby installed on your system, you can run the program as in the following:

```bash
docker run -it --rm -v $(pwd):/src -w /src helgecph/pythonruby sh -c "ruby gdbm_write.rb;ruby gdbm_read.rb"
```

What do all the Docker switches mean and do?

### Exercise 2

  * Inspect the file `fruitstore.db`
    - What is the output of `file fruitstore.db`?
    - What can you see when you look inside?
    - How does that compare to our earlier file from 
   
In case you do not have a tool like `xxd` or `hexdump` installed on your machine you can see the contents of the file for example with:
```bash
docker run -it --rm -v $(pwd):/src -w /src helgecph/pythonruby sh -c "hexdump -C fruitstore.db"
```

# Key-Value Stores for Production Systems

There are many types of key-value stores. They differ in if they are in-memory only, what type of disk they target, or the data types that can be used for keys and values.

  * Redis (https://redis.io) is an in-memory database. That is, it does not persist data into a file. Furthermore, it can handle complex data structures as values.
  * LevelDB (https://github.com/google/leveldb) from Google is an on disk key-value store, where both, keys and values, can be strings. It is similar to GDBM that we discussed earlier in that it is used as a library not as a client server system.
  * Find more systems listed here: https://en.wikipedia.org/wiki/Key-value_database or here: https://db-engines.com/en/ranking/key-value+store

A dumb Python GDBM implementation
https://github.com/python/cpython/blob/3.6/Lib/dbm/dumb.py

# Your turn at home!

![](http://www.twenty19.com/blog/wp-content/uploads/2017/07/typing2.gif)

## Assignment 1 - Simple DB with Hashmap-based Index

  * Build a `simple_db` in the programming language of your choice.
    - Implement a Hashmap-based index.
    - Implement functionality to store your data on disk in a binary file.
    - Implement functionality to read your data from disk again, so that you can reinstantiate your database after a shutdown.
  * _Hint:_ You do not want to serialize and deserialize the an in memory Hashmap containing all data directly. Instead, you in memory index based on a Hashmap contains information on where in you database file a piece of information is stored.

## Hand-in procedure

  * Provide all code and documentation for this assignment in a repository on Github.
  * Create a Markdown (.md) file called README.md in the root of your project.
  * That README.md describes what this project does and how to make it work. That is, you reviewer has to be able to clone your project, build it -you have to define steps for how to do that and what dependencies are required-, and use it.
  * Hand-in a link to your repository on www.peergrade.io.
  * Hand-in at latest on 5. Feb. 23:55.
  
## Review procedure

  * Log onto www.peergrade.io with your school email addresss.
  * Finish your review on www.peergrade.io at latest on 7. Feb. 12:00 (noon).
  * Make use of the review criteria below when giving feedback.
  * See the message that you received on your school email to know whom to review.
  

### Review Criteria

Provide feedback on each of the following bullet points.

  * Does the provided solution to the assignment work as described in the README.md?
    - If not, describe briefly what information is missing or what is wrong in the program.
  * Describe briefly, how the provided solution compares to the description of Hashmap-based indexes on page 4 of the PDF in https://github.com/datsoftlyngby/soft2018spring-databases-teaching-material/blob/master/literature/session_1.zip.
  * Does the provided solution allow for storage and reinstantiation of the Hashmap-based index when the database is shutdown and started respectively?
    - If not, provide information on what is missing and how that could be implemented.
